# RRN-LSTM with Tensorflow

Following the tutorial [A noob’s guide to implementing RNN-LSTM using Tensorflow](http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow).

See [Andrej Karpathy's blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) for more information on RRNs.

# Task
Given a binary string, containing `0`s and `1`s, of length 20, we need to determine the count of `1`s the string.

# Simple Soltuion
This is easy to solve without using a RRN, as shown in the code below. However, this is a simple problem with which to experiment using RRNs.

In [87]:
from functools import reduce

def count_num_1s(s: str) -> int:    
    return reduce((lambda acc, x: acc + x if x == 1 else acc), s)

# Should print out '9'.
print(str(count_num_1s([1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1])))

9


# RRN Solution

## Training Data

- Each input is a binary string of length 10.
- An input will be represented as a python list of `0`s and `1`s.
- There are $2^{10}$ possible combinations of `0`s and `1`s. 
- The network will be trained on all possible combinations.
- The inputs will be shuffled before giving them to the network to train with.

In [88]:
import numpy as np
from random import shuffle
from typing import List

def make_all_binary_strings(length: int = 10) -> List[List[int]]:
    """
    returns: a list of all possible binary strings, where a binary string is represened as a list of ints.
    """
    format_string = '{0:0' + str(length) + 'b}'
    all_binary_strings = [format_string.format(i) for i in range(2**length)]
    shuffle(all_binary_strings)
    return [list(map(int,i)) for i in all_binary_strings]

example_all = make_all_binary_strings(length=2)
print(example_all)

[[0, 0], [1, 0], [0, 1], [1, 1]]


- Tensorflow requires input as a tensor, i.e. a Tensorflow variable.
- The tensor is 3D with the dimensions [batch_size, sequence_length, input_dimension], where
    - `batch_size` is something we’ll determine later 
    - `sequence_length` is fixed at 10 
    - `input_dimension` is 1 i.e each individual bit of the string. Therefore, each bit will actually be represented as a list containing just that bit. 

In [89]:
def make_training(training_binary_strings: List[List[int]]) -> List[np.array]:
    # Turn every every element into an array containing that one element.
    # This is required as the input_dimension is 1.
    xs = list(map(lambda bits: list(map(lambda bit: [bit], bits)), training_binary_strings))
    # Make each array of bits a numpy array.
    return list(map(lambda bits: np.array(bits), xs))

example_training = make_training(example_all)
print(example_training)

[array([[0],
       [0]]), array([[1],
       [0]]), array([[0],
       [1]]), array([[1],
       [1]])]


### Labelling Training Data
- For every sequence, the result can be anything between 0 and 20. 
- Therefore there are have 21 categories in which a sequence can be classified into.
- Each sequence belongs to the class number which is the same as the count of ones in the sequence.
- The output is represented using 1-hot encoding.
- i.e. a vector containing all zeros except of one position which containings a one, indicating the class the sequence belongs to.

For example, the sequence `0011` would have the top output vector below, indicating there are two `1`s in the sequence.

`[0 0 1 0 0]`

`[0 1 2 3 4]` (the number of `1`s)

In [95]:
def label_training(training: List[np.array]) -> List[int]:
    def f(bits: np.array):
        # We made each element a single array in `make_training`.
        # By flattening we can more easily work with the data.
        flattened = bits.flatten()
        return count_num_1s(flattened)
    
    return list(map(f, training))

example_labels = label_training(example_training)
print(list(zip(example_all, example_labels)))

[([0, 0], 0), ([1, 0], 1), ([0, 1], 1), ([1, 1], 2)]
